<a href="https://colab.research.google.com/github/mrinaligupta255/Practice-on-Keras-Library/blob/master/Using_Dropout_in_Binary_classifcation_of_sonar_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dropout

Dropout is a technique where randomly selected neurons are ignored during training. They
are dropped-out randomly. This means that their contribution to the activation of downstream
neurons is temporally removed on the forward pass and any weight updates are not applied to
the neuron on the backward pass.
As a neural network learns, neuron weights settle into their context within the network.
Weights of neurons are tuned for specific features providing some specialization. Neighboring
neurons become to rely on this specialization, which if taken too far can result in a fragile model
too specialized to the training data. This reliant on context for a neuron during training is
referred to as complex co-adaptations. You can imagine that if neurons are randomly dropped
out of the network during training, that other neurons will have to step in and handle the
representation required to make predictions for the missing neurons. This is believed to result
in multiple independent internal representations being learned by the network.

In [0]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import numpy as np
import pandas as pd
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.pipeline import Pipeline

from keras.constraints import maxnorm

# to avoid update logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

seed=1
url='https://raw.githubusercontent.com/mrinaligupta255/Practice-on-Keras-Library/master/Binary%20Classification%20on%20Sonar%20Returns/sonar.csv'
dataframe=pd.read_csv(url,header=None)
data=dataframe.values
X=data[:,0:60]
Y=data[:,60]
encoder=LabelEncoder()
encoded_Y=encoder.fit_transform(Y)



In [0]:
def create_model_NO_DROPOUT():
  model=Sequential()
  model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu'))
  model.add(Dense(30,kernel_initializer='normal',activation='relu'))
  model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
  sgd=SGD(lr=0.01,momentum=0.8,decay=0.0,nesterov=False)
  model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])

  return model

def create_model_WITH_DROPOUT_ON_INPUT_LAYER():
  model=Sequential()
  model.add(Dropout(0.2,input_shape=(60,)))
  model.add(Dense(60,kernel_initializer='normal',activation='relu'))
  model.add(Dense(30,kernel_initializer='normal',activation='relu'))
  model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
  sgd=SGD(lr=0.01,momentum=0.8,decay=0.0,nesterov=False)

  model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])

  return model

def create_model_WITH_DROPOUT_ON_HIDDEN_LAYER():
    model=Sequential()
    model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30,kernel_initializer='normal',activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
    sgd=SGD(lr=0.01,momentum=0.8,decay=0.0,nesterov=False)
    model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])

    return model

In [3]:
#evaluate the baseline model with standardized dataset
#1. Without dropout
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_model_NO_DROPOUT,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
results=cross_val_score(pipeline,X,encoded_Y,cv=kfold)
print('Without dropout accuracy %.2f%% (%.2f%%)'% (results.mean()*100,results.std()*100))



Without dropout accuracy 84.12% (8.26%)


In [4]:
#2. Dropout in input layer
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_model_WITH_DROPOUT_ON_INPUT_LAYER,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
results=cross_val_score(pipeline,X,encoded_Y,cv=kfold)
print(' dropout in input layer accuracy %.2f%% (%.2f%%)'% (results.mean()*100,results.std()*100))


 dropout in input layer accuracy 85.54% (7.45%)


In [12]:
#3. Dropout in hidden layer
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_model_WITH_DROPOUT_ON_HIDDEN_LAYER,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
results=cross_val_score(pipeline,X,encoded_Y,cv=kfold)
print(' dropout in hidden layer accuracy %.2f%% (%.2f%%)'% (results.mean()*100,results.std()*100))

 dropout in hidden layer accuracy 84.59% (7.30%)
